# VANI

**Initial notebook configuration**

In [1]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [2]:
# Import VANI library
import dask
import numpy as np
import pandas as pd
from vani import Analyzer, ClusterOptions, ClusterType

**Initialize VANI Analyzer**

In [3]:
# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.LSF)
vn = Analyzer(n_workers=4, cluster_options=co, debug=True)

  0%|          | 0/3 [00:00<?, ?it/s]

{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': 'lassen708', 'dashboard_address': 'lassen708:8264', 'security': None}}
#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -G asccasc
#BSUB -q pdebug
#BSUB -W 02:00
#BSUB -o vani.log
#BSUB -e vani.log
JOB_ID=${LSB_JOBID%.*}

/usr/workspace/iopp/.conda/envs/jupyter/bin/python -m distributed.cli.dask_worker tcp://192.168.66.200:45231 --nthreads 1 --nprocs 4 --memory-limit 29.80GiB --name name --nanny --death-timeout 300

All 4 workers alive

**Analyze logs**

In [4]:
# Analysis configuration
log_dir = "/p/gpfs1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# Do the analysis
io_df_read_write, job_time = vn.analyze_parquet_logs(log_dir)

  0%|          | 0/7 [00:00<?, ?it/s]

Reading logs took 0.17106702644377947 seconds
Job time computation took 3.362377455458045 seconds


In [5]:
io_df_read_write.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_6,args_7,args_8,args_9,args_10,duration,filename,size,count,bandwidth
35,36,0,312784,0,18.552891,18.677382,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,,0.124491,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,514.094471
38,39,0,312784,0,18.677414,18.720806,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,,0.043392,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1474.920088
41,42,0,312784,0,18.720913,18.840509,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,,0.119596,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,535.132801
44,45,0,312784,0,18.840620,18.879042,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,,0.038422,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1665.728356
47,48,0,312784,0,18.879133,18.925989,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,,0.046856,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1365.889115


In [6]:
tbins = np.arange(0, job_time, 1)
tbins

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33.])

In [7]:
io_df_read_write["tbin"] = 0

for tbin in tbins:
    tstart_cond = io_df_read_write["tstart"].ge(tbin)
    tend_cond = io_df_read_write["tend"].lt(tbin + 1)
    io_df_read_write["tbin"] = io_df_read_write["tbin"].mask(tstart_cond & tend_cond, tbin)
    
io_df_read_write.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,filename,size,count,bandwidth,tbin
35,36,0,312784,0,18.552891,18.677382,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.124491,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,514.094471,18.0
38,39,0,312784,0,18.677414,18.720806,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.043392,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1474.920088,18.0
41,42,0,312784,0,18.720913,18.840509,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.119596,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,535.132801,18.0
44,45,0,312784,0,18.840620,18.879042,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.038422,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1665.728356,18.0
47,48,0,312784,0,18.879133,18.925989,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.046856,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1365.889115,18.0


In [8]:
%%time
def _size_bins(io_df_read_write):
    return io_df_read_write.groupby("tbin")["size"].sum()/1024.0/1024.0/1024.0

size_bins = _size_bins(io_df_read_write).compute()
size_bins.sort_values(ascending=False)

CPU times: user 1.66 s, sys: 53.2 ms, total: 1.71 s
Wall time: 5.49 s


tbin
23.0    236.734375
20.0    163.671875
0.0     130.828125
21.0    126.789062
11.0    120.742188
18.0     90.148438
15.0     75.773438
19.0     73.875000
22.0     71.640625
12.0     67.062500
17.0     61.921875
14.0     57.132812
24.0     54.500000
13.0     50.500000
16.0     47.937500
9.0      43.953125
10.0     24.312500
30.0     18.625000
5.0      16.250000
27.0     16.187500
25.0     10.281250
28.0      6.562500
26.0      4.546875
4.0       4.500000
29.0      3.625000
8.0       0.890625
6.0       0.703125
7.0       0.304688
Name: size, dtype: float64

In [9]:
%%time
def _bw_bins(io_df_read_write):
    return io_df_read_write.groupby("tbin")["bandwidth"].mean()/1024.0 

bw_bins = _bw_bins(io_df_read_write).compute()
bw_bins.sort_values(ascending=False)

CPU times: user 1.74 s, sys: 37.9 ms, total: 1.78 s
Wall time: 5.45 s


tbin
26.0    7.160398
16.0    5.554381
6.0     4.652142
14.0    4.552957
13.0    3.942404
15.0    3.848968
7.0     3.761858
21.0    3.651006
22.0    3.645205
19.0    3.640112
12.0    3.566132
17.0    3.560163
20.0    3.274907
28.0    3.267789
18.0    3.257141
11.0    3.202362
25.0    2.874886
10.0    2.740268
24.0    2.412552
8.0     2.356318
30.0    2.353486
23.0    2.269931
27.0    2.162386
9.0     1.782753
29.0    1.651270
5.0     1.358728
0.0     0.854868
4.0     0.809555
Name: bandwidth, dtype: float64

In [10]:
%%time
size_bins, bw_bins = dask.compute(_size_bins(io_df_read_write), _bw_bins(io_df_read_write))

CPU times: user 1.86 s, sys: 53.3 ms, total: 1.92 s
Wall time: 5.63 s


In [11]:
size_bins_cats = pd.cut(size_bins.sort_values(ascending=False), bins=3, labels=["low", "medium", "high"])
size_bins_cats

tbin
23.0      high
20.0      high
0.0     medium
21.0    medium
11.0    medium
18.0    medium
15.0       low
19.0       low
22.0       low
12.0       low
17.0       low
14.0       low
24.0       low
13.0       low
16.0       low
9.0        low
10.0       low
30.0       low
5.0        low
27.0       low
25.0       low
28.0       low
26.0       low
4.0        low
29.0       low
8.0        low
6.0        low
7.0        low
Name: size, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']

In [12]:
bw_bins_cats = pd.cut(bw_bins.sort_values(ascending=False), bins=3, labels=["low", "medium", "high"])
bw_bins_cats

tbin
26.0      high
16.0      high
6.0     medium
14.0    medium
13.0    medium
15.0    medium
7.0     medium
21.0    medium
22.0    medium
19.0    medium
12.0    medium
17.0    medium
20.0    medium
28.0    medium
18.0    medium
11.0    medium
25.0       low
10.0       low
24.0       low
8.0        low
30.0       low
23.0       low
27.0       low
9.0        low
29.0       low
5.0        low
0.0        low
4.0        low
Name: bandwidth, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']

In [13]:
problematic_bins = size_bins[size_bins_cats.eq(bw_bins_cats) == False]
problematic_bins.sort_values(ascending=False)

tbin
23.0    236.734375
20.0    163.671875
0.0     130.828125
15.0     75.773438
19.0     73.875000
22.0     71.640625
12.0     67.062500
17.0     61.921875
14.0     57.132812
13.0     50.500000
16.0     47.937500
28.0      6.562500
26.0      4.546875
6.0       0.703125
7.0       0.304688
Name: size, dtype: float64

In [14]:
io_df_read_write_bin_23 = io_df_read_write[io_df_read_write["tbin"] == 23].compute()
io_df_read_write_bin_23.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,filename,size,count,bandwidth,tbin
35,36,40,312784,0,23.414318,23.455755,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.041437,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,1544.507802,23.0
38,39,40,312784,0,23.455805,23.466869,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.011065,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,5784.249612,23.0
41,42,40,312784,0,23.466919,23.476877,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.009958,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,6426.820916,23.0
44,45,40,312784,0,23.476931,23.487055,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.010124,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,6321.483044,23.0
47,48,40,312784,0,23.487112,23.498230,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.011118,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,5756.464574,23.0


In [15]:
io_df_read_write_bin_23.describe()

,index,rank,thread_id,cat,tstart,tend,level,arg_count,duration,size,count,bandwidth,tbin
count,3855.000000,3855.000000,3855.0,3855.0,3855.000000,3855.000000,3855.0,3855.0,3855.000000,3.855000e+03,3855.0,3855.000000,3855.0
mean,536.764462,632.805707,312784.0,0.0,23.455437,23.520262,0.0,3.0,0.064849,6.593816e+07,1.0,2324.409521,23.0
std,292.310669,315.215281,0.0,0.0,0.259748,0.261142,0.0,0.0,0.080231,2.584622e+07,0.0,1567.257176,0.0
min,36.000000,40.000000,312784.0,0.0,23.000019,23.011869,0.0,3.0,0.002481,2.516582e+07,1.0,116.704979,23.0
25%,239.000000,413.000000,312784.0,0.0,23.231423,23.306331,0.0,3.0,0.016753,6.710886e+07,1.0,763.647288,23.0
50%,493.000000,603.000000,312784.0,0.0,23.462048,23.528444,0.0,3.0,0.027031,6.710886e+07,1.0,2504.996790,23.0
75%,747.000000,889.000000,312784.0,0.0,23.656260,23.730779,0.0,3.0,0.082095,6.710886e+07,1.0,3546.792912,23.0
max,1001.000000,1239.000000,312784.0,0.0,23.992842,23.999588,0.0,3.0,0.548391,1.342177e+08,1.0,9671.723290,23.0


In [16]:
io_df_read_write_bin_23.memory_usage().sum()

663060

In [17]:
unique_files = io_df_read_write_bin_23["filename"].unique()
len(unique_files)

521

In [18]:
io_df_read_write_bin_23["filename"]

35     /p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...
38     /p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...
41     /p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...
44     /p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...
47     /p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...
                             ...                        
963    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001239-...
975    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001239-...
978    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001239-...
981    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001239-...
984    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001239-...
Name: filename, Length: 3855, dtype: object

In [19]:
filename_bws = io_df_read_write_bin_23.groupby("filename")["bandwidth"].mean()/1024.0
filename_bws.describe()

count    521.000000
mean       2.117531
std        0.970718
min        0.119582
25%        1.592594
50%        2.137641
75%        2.631433
max        6.092821
Name: bandwidth, dtype: float64

In [20]:
filename_bws_mean = filename_bws.mean()*1024.0
filename_bws_std = filename_bws.std()*1024.0
filename_bws_min = filename_bws.min()*1024.0
filename_bws_mean, filename_bws_std, filename_bws_min

(2168.3516781815056, 994.0151953958358, 122.45189668930887)

In [21]:
bw_cond_start = io_df_read_write_bin_23["bandwidth"] >= filename_bws_min
bw_cond_end = io_df_read_write_bin_23["bandwidth"] < (filename_bws_min+filename_bws_std)
io_df_read_write_bin_23_low_bw = io_df_read_write_bin_23[bw_cond_start & bw_cond_end]
io_df_read_write_bin_23_low_bw.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,filename,size,count,bandwidth,tbin
50,51,40,312784,0,23.498287,23.651049,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.152761,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,418.953840,23.0
53,54,40,312784,0,23.651083,23.786854,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,...,,,,,0.135771,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000040-...,67108864.0,1.0,471.382662,23.0
817,818,78,312784,0,23.603487,23.749922,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000078-...,...,,,,,0.146435,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000078-...,134217728.0,1.0,874.109256,23.0
329,330,82,312784,0,23.852644,23.909096,read,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,...,,,,,0.056452,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,25165824.0,1.0,425.141467,23.0
837,838,86,312784,0,23.884684,23.967989,read,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000086-...,...,,,,,0.083305,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000086-...,25165824.0,1.0,288.096712,23.0


In [22]:
unique_files_low_bw = io_df_read_write_bin_23_low_bw["filename"].unique()
len(unique_files_low_bw)

452

In [23]:
# all_accessed_by_same_rank
io_df_read_write_bin_23_low_bw.groupby("filename")["rank"].nunique().all()

True

In [24]:
# all_accessed_by_thread_id
io_df_read_write_bin_23_low_bw.groupby("filename")["thread_id"].nunique().all()

True

In [25]:
# all_accessed_by_func_id
io_df_read_write_bin_23_low_bw.groupby("filename")["func_id"].nunique().all()

True

Time > Timeline on I/O size > Timeline I/O bw > Most important files > *Transfer sizes* > *Parallelism of ops*
Find the epsilon from mean of durations > how many ranks were doing I/O > 1 second/bins
They should be ~1280 = High parallelism is the problem 